<a href="https://colab.research.google.com/github/Mathias1232/comp215/blob/main/labs/lab02-review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 2
----------------
#### Name(s):
#### Date:

By the end of this lab you should be able to:
  * use strings, tuples, lists, and dictionaries (review)
  * use *f-string* to simplify string formatting operations
  * write functions and simple unit tests (review)
  * use *list comprehension* to implement map and filter algorithms


During this lab, you will be introduced to the following:
  * API Keys
  * JSON data formats
  * the `datetime` module to represent dates and times
  * the `pprint` module to format data structures

## Near Earth Objects

In this lab, we'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/) using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS).  But, as usual, let's do the imports first.


In [ ]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

### dates & f-strings

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [ ]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the string
print(formatted_date)

### Make an API Query

Let's get some data from the NEO database; here's a query that gets the observation "feed" for today.  Notice the first line of the block is an 'API key'.  An API key is a unique identifier that authenticates a user when making a request to an API.  Run the block below using the API key 'DEMO_KEY' (just to be sure it works), then go to https://api.nasa.gov and get your own API key.  Use your own API key for the remainder of the lab.

In [ ]:
API_KEY = 'zvONe8dKAGJF1mlxEGM2Ae0ufJJ7S7T2cDq7hH5J'  # substitute your API key here

# get today's date (as a string)
#today = str(datetime.date.today())
todayGlobal = '2023-01-09'

# use an f-string to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={todayGlobal}&end_date={todayGlobal}&api_key={API_KEY}'

# make the API request (recall from lab 1)
response = requests.request("GET", url, headers={}, data={})

# convert the response to json format (recall from lab 1)
data = json.loads(response.text)

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neosGlobal = data['near_earth_objects'][todayGlobal]
print(f'{n_results} Near Earth Objects found for {todayGlobal}')
pprint(neosGlobal)

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [ ]:
hazards =  []
for item in neosGlobal:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')


### Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.  We can use this `id` to fetch complete orbital and close approach data for the NEO.  For example, this query fetches the complete data set for the first hazardous asteroid...  Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".


In [ ]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

## Exercise 1

In the code cell below, **re-write the accumulator loop** that creates the list of hazards as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [22]:
# Ex. 1 your code here

#hazards = list(filter(lambda item: item['is_potentially_hazardous_asteroid'] is True, neos))

# Makes a list using list comprehension using an if to filter the hazardous objects
hazards = [item for item in neosGlobal if item['is_potentially_hazardous_asteroid']]
print(f'{len(hazards)} potentially hazardous asteroids identified.')


# For Assert
# a is the hardcoded example for neos list
a = [
    {'is_potentially_hazardous_asteroid':True, 'id':'a', 'close_approach_data': [{'close_approach_date': '2024-01-09', 'miss_distance': {'kilometers': '2'}}]},
    {'is_potentially_hazardous_asteroid':True, 'id':'b', 'close_approach_data': [{'close_approach_date': '2023-01-09', 'miss_distance': {'kilometers': '3'}}]},
    {'is_potentially_hazardous_asteroid':False, 'id':'c', 'close_approach_data': [{'close_approach_date': '2022-01-09', 'miss_distance': {'kilometers': '4'}}]},
    {'is_potentially_hazardous_asteroid':True, 'id':'d', 'close_approach_data': [{'close_approach_date': '2021-01-09', 'miss_distance': {'kilometers': '5'}}]}
    ]
# b performs the same list comprehension as earlier to filter out non hazardous objects
b = [item for item in a if item['is_potentially_hazardous_asteroid']]
# c is the hardcoded filtered list
c= [
    {'is_potentially_hazardous_asteroid':True, 'id':'a', 'close_approach_data': [{'close_approach_date': '2024-01-09', 'miss_distance': {'kilometers': '2'}}]},
    {'is_potentially_hazardous_asteroid':True, 'id':'b', 'close_approach_data': [{'close_approach_date': '2023-01-09', 'miss_distance': {'kilometers': '3'}}]},
    {'is_potentially_hazardous_asteroid':True, 'id':'d', 'close_approach_data': [{'close_approach_date': '2021-01-09', 'miss_distance': {'kilometers': '5'}}]}
    ]
# This asserts that the list comprehension and filter correctly makes a new list with only the hazardous objects
assert b == c

3 potentially hazardous asteroids identified.




## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"


Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [21]:
# Ex. 2 your code here
def closestApproach(data):
  '''
  Finds the closest hazardous approach an object in a given list has had with earth, as long as the list has data

  Args:
    data (list): A list of dictionaries of different object encounters

  Returns:
    Tuple: Returns a tuple with format (Date, Distance in km, id)
  '''

  # Checks if data is empty or not
  if data:
    # Sets the closest approach distance, date, and id to that of the first hazard in the list
    closestDistance = data[0]['close_approach_data'][0]['miss_distance']['kilometers']
    closestDate = data[0]['close_approach_data'][0]['close_approach_date']
    closestID = data[0]['id']
    # Loops through the rest of the hazards in the list not including the first and compares their distances
    # If the distance of the new object is smaller than sets that as the new closest distance, also updates the date and id
    for hazard in data[1:]:
      currentDistance = hazard['close_approach_data'][0]['miss_distance']['kilometers']
      currentDate = hazard['close_approach_data'][0]['close_approach_date']
      currentID = hazard['id']
      if currentDistance < closestDistance:
        closestDistance = currentDistance
        closestDate = currentDate
        closestID = currentID
    return (closestDate, float(closestDistance), closestID)

# Assert using the previously made test list
assert closestApproach(b) == ('2024-01-09', 2.0, 'a')

## Challenge Exercise

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [23]:
# Ex. 3 (challenge) your code here
API_KEY = 'zvONe8dKAGJF1mlxEGM2Ae0ufJJ7S7T2cDq7hH5J'
from datetime import timedelta
def closestThisWeek():
  '''
  Finds the closest approach an object has had with earth in the past week, returns a tuple with date and distance of encounter

  Args:
    None

  Returns:
    Tuple: A tuple with the date and distance of the hazardous encounter format (date, distance in km)
  '''

  # Get today's date
  today = str(datetime.date.today())
  # Get the date a week ago
  weekAgo = str(datetime.date.today() - timedelta(days=7))
  # format the date and API key varaibles.
  url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={weekAgo}&end_date={today}&api_key={API_KEY}'
  # make the API request (recall from lab 1)
  response = requests.request("GET", url, headers={}, data={})
  # convert the response to json format
  data = json.loads(response.text)
  neos = data['near_earth_objects']
  hazards = []

  # Given the neos data is sorted into days, this loop compiles all the data from different days into one list to be sorted by the closestApproach function
  for date in neos.keys():
    hazards += [item for item in neos[date] if item['is_potentially_hazardous_asteroid']]

  closestDistance = closestApproach(hazards)

  return closestDistance

closestThisWeek()

('2025-01-14', 17371524.962223362, '2506491')